In [49]:
import os
from pdf2image import convert_from_path
from PIL import Image
import requests
import sqlite3
from datetime import datetime


In [50]:
def files_path(folder_path):

    pdf_files = []

    for file in os.listdir(folder_path):
        full_path = os.path.join(folder_path, file)
        if os.path.isfile(full_path) and file.endswith('.pdf'):
            pdf_files.append(full_path)
            
    return pdf_files

In [51]:
def make_front_page_image(book_path, folder_path):

    images = convert_from_path(book_path, first_page=1, last_page=1)
    
    image_path = os.path.join(folder_path, 'front_page.png')
    images[0].save(image_path, 'PNG')

    return image_path



In [52]:
def sent_book_ang_front_page(file_path, image_path):
    TOKEN = '7791673517:AAGWdQPwgtxcTPKeLUJzVJtUqBT-pI6pJhU'
    CHANNEL_ID = '@tursunov_Books'
    
    url_send_document = f'https://api.telegram.org/bot{TOKEN}/sendDocument'
    url_send_photo = f'https://api.telegram.org/bot{TOKEN}/sendPhoto'

    with open(file_path, 'rb') as file:

        response = requests.post(url_send_document, data={'chat_id': CHANNEL_ID,}, files={'document': file})
        message_id = response.json()['result']['message_id']


    with open(image_path, 'rb') as img_file:

        response = requests.post(url_send_photo, data={
            'chat_id': CHANNEL_ID,
            'reply_to_message_id': message_id,
        }, files={'photo': img_file})
    if response.status_code == 200:
        return os.path.basename(file_path),'Successfully', datetime.now()
    else:
        print(f'Error accurred: {response.text}')


In [53]:
def create_data_base():
    with sqlite3.connect('demo.db') as data_base:

#    with sqlite3.connect('books_list.db') as data_base:
    
        cursor = data_base.cursor()
        query_create_table = 'CREATE TABLE IF NOT EXISTS About_books (ID integer, Name text, Operation_report text, Date_time text)'
        cursor.execute(query_create_table)

In [54]:
def insert_data(id_book, name_book, report, in_time):
    with sqlite3.connect('demo.db') as data_base:
    
#    with sqlite3.connect('books_list.db') as data_base:
    
        cursor = data_base.cursor()
        insert_query = 'INSERT INTO About_books VALUES(?, ?, ?, ?)'
        info = [(id_book, name_book, report, in_time), ]
        cursor.executemany(insert_query, info)

In [55]:
def main():
    Image.MAX_IMAGE_PIXELS = None
    folder_path = r'D:\course_from_maab\project\books'
    #folder_path = r'D:\course_from_maab\project\mini_books'
    pdf_files = files_path(folder_path)

    create_data_base()
    id = 0
    for pdf_path in pdf_files:
        id += 1
        image_path = make_front_page_image(pdf_path, folder_path)
        name_book, report, in_time = sent_book_ang_front_page(pdf_path, image_path)
        insert_data(id, name_book, report, in_time)
        os.remove(image_path)

In [56]:
if __name__ == '__main__':
    main()